## Building a symbolic classifier

In [5]:
import skmine
import pandas as pd
print("This tutorial was tested with the following version of skmine :", skmine.__version__)


This tutorial was tested with the following version of skmine : 0.0.9


MDL based algorithms encode data according to a given codetable

When calling ``.fit``, we iteratively look for the codetable that compress
the training data the best

**When we are done with training our model, we can benefit from the refined codetable 
to make some predictions**

### SLIM Classifier

An **integrated classifier in scikit mine is available** and allows to **solve binary and multiclass problems**. It uses the SLIM compression algorithm. 

To use it, we need to have **discretized dataset**. Let's take for example the **discretized iris dataset**.

In [19]:
from skmine.datasets.fimi import fetch_iris

X, y = fetch_iris(return_y=True)  # without return_y=True, the method would have returned the whole dataset in one variable
from sklearn.datasets import load_iris
target_names = ['setosa', 'versicolor', 'virginica']
print(pd.DataFrame(X))
print(pd.DataFrame(y))

    iris.D19.N150.C3.num
0         [2, 9, 12, 15]
1        [1, 10, 11, 14]
2        [5, 10, 13, 16]
3         [2, 6, 12, 15]
4         [1, 8, 11, 14]
..                   ...
145       [3, 9, 13, 16]
146      [1, 10, 11, 14]
147       [3, 8, 12, 15]
148       [5, 9, 13, 16]
149      [5, 10, 13, 16]

[150 rows x 1 columns]
     iris.D19.N150.C3.num
0                      18
1                      17
2                      19
3                      18
4                      17
..                    ...
145                    19
146                    17
147                    18
148                    19
149                    19

[150 rows x 1 columns]


The purpose of this dataset is to **predict the last column of db from the other 4**. The possible targets are: 17, 18, 19. We can prepare our train and test data set.

In [20]:
from sklearn.model_selection import train_test_split

(X_train, X_test, y_train, y_test) = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=True)
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

X_train shape: (120,) y_train shape: (120,)
X_test shape: (30,) y_test shape: (30,)


Now we can use our **SlimClassifier**.

In [21]:
from skmine.itemsets.slim_classifier import SlimClassifier

# You can pass in parameter of your classifier the set of your items.  This will improve its performance especially on small data sets like iris.
items = set(item for transaction in X for item in transaction)

clf = SlimClassifier(items=items)  # You can also enable or disable the pruning of SLIM compressors via the `pruning` parameter
clf.fit(X_train, y_train)

SlimClassifier(items={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16})

You can use many functions of sckit learn that are compatible with classifiers. For example, build a **confusion matrix**, use **GridSearchCV** or **cross validation**.

- **Confusion matrix**

In [22]:
from sklearn.metrics import confusion_matrix

y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)
print(f"Validation accuracy :{round(clf.score(X_test, y_test) * 100,1)} %")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), columns=target_names, index=target_names))

Validation accuracy :83.3 %
            setosa  versicolor  virginica
setosa          13           1          0
versicolor       0           8          1
virginica        0           3          4


- **GridSearchCV** (this method allows us to test many parameters for a classifier and to retain the best combination)

In [23]:
from sklearn.model_selection import GridSearchCV

parameters = {'pruning': [False, True], 'items': [None, items]}
grid = GridSearchCV(clf, parameters)
grid.fit(X_train,y_train)
print(grid.best_params_)
print(grid.score(X_train, y_train))

{'items': {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}, 'pruning': False}
0.9833333333333333


With GridSearchCV we get with the best parameters an **accuracy of more than 98%**, much better than the previous score. With this combination, the item list is passed as a parameter and pruning is disabled. Since pruning does not improve the compression of codetables in the SLIM algorithm on iris, it does not matter whether it is enabled or not.

To reduce overfitting, we can use the cross validation of sklearn.
- **Cross validation**

In [48]:
from sklearn.model_selection import cross_val_score

cross_validation = cross_val_score(clf, X, y, cv=10)
print(f"Cross validation accuracies: {np.round(cross_validation,2)} MEAN : {round(cross_validation.mean()*100,1)} %.  ")


Cross validation accuracies: [0.93 0.93 0.87 0.93 0.93 0.93 1.   1.   1.   0.93] MEAN : 94.7 %.  


After cross validation, we see that the **accuracy is almost 95% on average**. So in 95% of the cases, the right type of flower is given.

----------

### OneVsRest classifier

The **SLIM algorithm** is also compatible with **scikit-learn** to be used from other classifiers like **One-vs-the-rest (OvR)** (https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html). The limitation of this method is that the classifier **only works for multiclass classification** problems while the embedded classifier works for both binary and multiclass problems.

In [49]:
from skmine.itemsets import SLIM
from sklearn.preprocessing import MultiLabelBinarizer

In [50]:
class TransactionEncoder(MultiLabelBinarizer):  # pandas DataFrames are easier to read ;)
    def transform(self, X):
        _X = super().transform(X)
        return pd.DataFrame(data=_X, columns=self.classes_)

In [51]:
transactions = [ 
     ['bananas', 'milk'], 
     ['milk', 'bananas', 'cookies'], 
     ['cookies', 'butter', 'tea'], 
     ['tea'],  
     ['milk', 'bananas', 'tea'], 
]
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D 

,bananas,butter,cookies,milk,tea
0,1,0,0,1,0
1,1,0,1,1,0
2,0,1,1,0,1
3,0,0,0,0,1
4,1,0,0,1,1


In [52]:
slim = SLIM()
slim.fit(D).transform(D)

,itemset,usage
0,"[bananas, milk]",3
1,[tea],3
2,[cookies],2
3,[butter],1


We keep this **codetable** in mind, as we will later use it **to interpret our predictions**

----------
#### First "predictions" 

We start from last SLIM fitted model to look at new itemsets, gathered in `new_transactions`. I we look at new itemset $x$=`['milk', 'butter', 'cookies']`

In [60]:
new_transactions = [ 
   ['milk', 'sirup', 'cookies'], 
   ['butter', 'tea'], 
   ['bananas', 'milk'],  
]
new_D = te.transform(new_transactions)
new_D

/home/hcourtei/miniconda3/envs/ml-sklearn/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['sirup'] will be ignored
  warnings.warn(


,bananas,butter,cookies,milk,tea
0,0,0,1,1,0
1,0,1,0,0,1
2,1,0,0,1,0


Once slim is fitted on `transaction`, a codetable is inferred from training data. Thanks to it,  we can compute code lentgh $c_l(x)$  for $x$  with method `.get_code_length` ($c_l \in [0, +\infty[$).

Shortest code length  means that $x$ is close to fitted dataset.

So, we transform codes length by method  `.decision_function`  into a probability $score \in [0,1]$ to belong to same dataset, that will help us to **classify itemset**. Currently a negative exponential is used $ score = \exp(-0.2 \times c_l)$ .


In [83]:
codes_length = slim.get_code_length(new_D)
scores_slim = slim.decision_function(new_D)
pd.DataFrame([new_transactions, codes_length, scores_slim], index=['transaction', 'codes_length','scores_slim']).T

,transaction,codes_length,scores_slim
0,"[milk, sirup, cookies]",6.228819,0.287721
1,"[butter, tea]",4.813781,0.381839
2,"[bananas, milk]",1.906891,0.68292



First observations

* itemset `[milk, sirup, cookies] ` has the highest code length, so the smallest score for decision function.From the codetable we see `milk` and `cookies` are not    grouped together, while `sirup` has never been seen
  

* `[bananas, milk]` has the lowest code length, so the highest score. It contains `bananas` and `milk`, which are grouped together in the codetable and have high occurence in the training data.

#### Shortest code wins !!
Next, we are going to use an ensemble of SLIM encoding schemes, and utilize them via a ``OneVsRest`` methodology, to perform **multi-class classification**.
The methodology is very simple

1. Clone our base estimator as many time as we need (one per class)
2. Fit every estimator on entries corresponding to its class in the input data
3. When calling ``.predict``, we actually call ``.decision_function`` and get score for every class
4. The shorted code wins : we choose the class with the lowest code length (so the highest score) for a given transaction

In [62]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

In [63]:
pipe = Pipeline([
    ('transaction_encoder', TransactionEncoder(sparse_output=False)),
    ('slim', SLIM()),
])

In [64]:
transactions = [
    ['milk', 'bananas'],
    ['tea', 'New York Times', 'El Pais'],
    ['New York Times'],
    ['El Pais', 'The Economist'],
    ['milk', 'tea'],
    ['croissant', 'tea'],
    ['croissant', 'chocolatine', 'milk'],
]
target = [
    'foodstore', 
    'newspaper', 
    'newspaper', 
    'newspaper', 
    'foodstore',
    'bakery',
    'bakery',
]

In [65]:
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D

,El Pais,New York Times,The Economist,bananas,chocolatine,croissant,milk,tea
0,0,0,0,1,0,0,1,0
1,1,1,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0
4,0,0,0,0,0,0,1,1
5,0,0,0,0,0,1,0,1
6,0,0,0,0,1,1,1,0


In [66]:
ovr = OneVsRestClassifier(SLIM())

In [67]:
ovr.fit(D, y=target)
ovr.estimators_

[SLIM(), SLIM(), SLIM()]

In [68]:
pd.DataFrame(ovr.decision_function(D), columns=ovr.classes_)

,bakery,foodstore,newspaper
0,0.238357,0.399719,0.218069
1,0.116370,0.142135,0.234470
2,0.488218,0.488218,0.641159
3,0.238357,0.238357,0.365697
4,0.238357,0.399719,0.266351
5,0.596311,0.291130,0.266351
6,0.596311,0.159776,0.101834


In [69]:
ovr.predict(D)

array(['foodstore', 'newspaper', 'newspaper', 'newspaper', 'foodstore',
       'bakery', 'bakery'], dtype='<U9')

#### Questions on binary OneVsRest classifier

In [70]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('transaction_encoder', TransactionEncoder(sparse_output=False)),
    ('slim', SLIM()),
])

In [71]:
transactions = [
    ['milk', 'bananas'],
    ['tea', 'New York Times', 'El Pais'],
    ['New York Times'],
    ['El Pais', 'The Economist'],
    ['milk', 'tea'],
]
target = [
    'foodstore', 
    'newspaper', 
    'newspaper', 
    'newspaper', 
    'foodstore',
]

In [72]:
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D

,El Pais,New York Times,The Economist,bananas,milk,tea
0,0,0,0,1,1,0
1,1,1,0,0,0,1
2,0,1,0,0,0,0
3,1,0,1,0,0,0
4,0,0,0,0,1,1


In [73]:
ovr = OneVsRestClassifier(SLIM())

In [74]:
ovr.fit(D, y=target)
ovr.estimators_

[SLIM()]

In [76]:
ovr.decision_function(D)

0    0.238357
1    0.267940
2    0.670320
3    0.399719
4    0.291130
dtype: float32

**=> Which thresold can we take to decide if the prediction is 'foodstore', or 'newspaper' ?**

#### OneVsOne

In [77]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.pipeline import Pipeline

In [78]:
pipe = Pipeline([
    ('transaction_encoder', TransactionEncoder(sparse_output=False)),
    ('slim', SLIM()),
])

In [79]:
transactions = [
    ['milk', 'bananas'],
    ['tea', 'New York Times', 'El Pais'],
    ['New York Times'],
    ['El Pais', 'The Economist'],
    ['milk', 'tea'],
    ['croissant', 'tea'],
    ['croissant', 'chocolatine', 'milk'],
    ['Monde diplo', 'Game of Thrones', 'Harry Potter'],
    ['New York Times', 'Harry Potter'],
]
target = [
    'foodstore', 
    'newspaper', 
    'newspaper', 
    'newspaper', 
    'foodstore',
    'bakery',
    'bakery',
    'library',
    'library',
]

In [80]:
te = TransactionEncoder()
D = te.fit(transactions).transform(transactions)
D

,El Pais,Game of Thrones,Harry Potter,Monde diplo,New York Times,The Economist,bananas,chocolatine,croissant,milk,tea
0,0,0,0,0,0,0,1,0,0,1,0
1,1,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,1
5,0,0,0,0,0,0,0,0,1,0,1
6,0,0,0,0,0,0,0,1,1,1,0
7,0,1,1,1,0,0,0,0,0,0,0
8,0,0,1,0,1,0,0,0,0,0,0


In [81]:
ovo = OneVsOneClassifier(SLIM())

In [82]:
ovo.fit(D, y=target)
ovo.estimators_

(SLIM(), SLIM(), SLIM(), SLIM(), SLIM(), SLIM())

In [39]:
ovo.decision_function(D)

AttributeError: 'SLIM' object has no attribute 'predict'

In [ ]:
ovr.predict(D)

AttributeError: 'SLIM' object has no attribute 'predict'

In [ ]:
pd.DataFrame(ovr.decision_function(D), columns=ovr.classes_)

In [ ]:
>>> from sklearn.datasets import load_iris
>>> from sklearn.model_selection import train_test_split
>>> from sklearn.multiclass import OneVsOneClassifier
>>> from sklearn.svm import LinearSVC
>>> X, y = load_iris(return_X_y=True)
>>> X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.33, shuffle=True, random_state=0)
>>> clf = OneVsOneClassifier(
...     LinearSVC(random_state=0)).fit(X_train, y_train)
>>> clf.predict(X_test[:10])

/home/cregan/fsl/lib/python3.10/site-packages/scikit_learn-1.2.0-py3.10-linux-x86_64.egg/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


array([2, 1, 0, 2, 0, 2, 0, 1, 1, 1])